In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy
import requests
from config import weather_api_keys
from datetime import datetime

In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(-90.000, 90.000, 1500)
lngs = np.random.uniform(-180.000, 180.000, 1500)
lats_lngs = zip(lats,lngs)

In [3]:
# Add the latitudes and longitudes to a list.
coordinates = list(lats_lngs)

In [4]:
# Create a list for holding the cities.
cities = []
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0],coordinate[1]).city_name
    
    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
        
# Print the city count to confirm sufficient count.
len(cities)
    

614

In [6]:
# Create an empty list to hold the weather data.
url = "https://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_keys
city_data =[]
# Print the beginning of the logging.
print("Beginning Data Retrieval     \n")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in our list.
for i, city in enumerate(cities):
    #Group cities in sets of 50 for loggin purposes.
    if (i%50==0 and i>=50):
        set_count+=1
        record_count =1
# Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(' ','+')
    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    try:
         # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})
    # If an error is experienced, skip the city.
    except:
        print(f'{city} not found. Skipping...')
        pass
        
# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")
    

Beginning Data Retrieval     

-----------------------------
Processing Record 1 of Set 1 | mataura
Processing Record 2 of Set 1 | port elizabeth
Processing Record 3 of Set 1 | busselton
Processing Record 4 of Set 1 | puerto ayora
Processing Record 5 of Set 1 | saldanha
Processing Record 6 of Set 1 | cabinda
Processing Record 7 of Set 1 | rikitea
Processing Record 8 of Set 1 | punta arenas
Processing Record 9 of Set 1 | bonavista
Processing Record 10 of Set 1 | smithers
Processing Record 11 of Set 1 | yulara
Processing Record 12 of Set 1 | black river
Processing Record 13 of Set 1 | itarema
Processing Record 14 of Set 1 | kamenskoye
Cirty not found. Skipping...
Processing Record 15 of Set 1 | cabo san lucas
Processing Record 16 of Set 1 | hermanus
Processing Record 17 of Set 1 | ushuaia
Processing Record 18 of Set 1 | san policarpo
Processing Record 19 of Set 1 | tumannyy
Cirty not found. Skipping...
Processing Record 20 of Set 1 | vaini
Processing Record 21 of Set 1 | nemuro
Processin

In [7]:
len(city_data)

569

In [8]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Mataura,-46.19,168.86,35.01,95,100,10.00,NZ,2020-09-27 19:17:10
1,Port Elizabeth,-33.92,25.57,51.80,71,0,7.54,ZA,2020-09-27 19:17:10
2,Busselton,-33.65,115.33,53.01,88,1,9.31,AU,2020-09-27 19:17:11
3,Puerto Ayora,-0.74,-90.35,71.01,86,82,8.01,EC,2020-09-27 19:17:11
4,Saldanha,-33.01,17.94,50.00,71,0,4.70,ZA,2020-09-27 19:17:12
5,Cabinda,-5.55,12.20,75.27,77,70,7.11,AO,2020-09-27 19:17:12
6,Rikitea,-23.12,-134.97,74.32,81,2,12.44,PF,2020-09-27 19:17:13
7,Punta Arenas,-53.15,-70.92,51.80,50,75,23.04,CL,2020-09-27 19:17:13
8,Bonavista,48.65,-53.11,62.24,74,100,22.37,CA,2020-09-27 19:17:13
9,Smithers,54.78,-127.17,44.60,93,90,8.05,CA,2020-09-27 19:17:14


In [10]:
new_column_order = ['City','Country', 'Date', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness', 'Wind Speed']
city_data_df =city_data_df[new_column_order]

In [13]:
# Create the output file (CSV).
output_data_file = 'weather_data/cities.csv'
city_data_df.to_csv(output_data_file, index_label ='City_ID')